In [ ]:
import os
import subprocess
import logging
import glob
import random
import string
import csv
import shutil
import pandas as pd
import openpyxl



# Setup logging
logging.basicConfig(
    filename="bids_conversion.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

os.chdir("/media/administrator/Scratch/dcm2bids-cpp/code")

## Set your DICOM and BIDS path (change as needed)

In this cell, you will find the path that you need to set up for the scan conversion, such as
- The path where you store your DICOM files (named : dicom_dir)
- The folder where you want you BIDS files to be stored (named : bids_dir)
- The json file that dcm2bids will use to know which file to convert (named : config)
- A path to an excel file that includes 1.the alphanumerical number for each participant on the left column as well as 2.the coded name of the participant (CPP000...) on the right column (named : excel_mapping_file) 

In [127]:
# Define your paths here 
dicom_dir = '/media/administrator/Scratch/dcm2bids-cpp/code/DICOM_ALLEN'
bids_dir = "/media/administrator/Scratch/dcm2bids-cpp/code/BIDS_final"
config = '/media/administrator/Scratch/dcm2bids-cpp/code/dcm2bids_config.json'
session = "01"   # ex: "01", "02", etc.
excel_mapping_file = '/media/administrator/Scratch/dcm2bids-cpp/code/CPP_Dual_Coding_DataRelease2.xlsx'



## Helper function ##
This cell has been made to create custom functions that will be used later for the conversion of all our DICOM into BIDS format, such as
- run_dcm2bids
- run_pydeface
- run_bidsphysio

In [128]:
def run_dcm2bids(orig_id, anon_id, dicom_dir, bids_dir, config, session="1"):
    """
    Convert DICOM to BIDS if not already converted.
    """
    subject_bids_dir = os.path.join(bids_dir, f"sub-{anon_id}", f"ses-{session}")

    # ✅ Skip if participant/session already converted
    if os.path.exists(subject_bids_dir) and any(os.scandir(subject_bids_dir)):
        logging.info(f"✅ Participant {anon_id} (session {session}) already converted. Skipping dcm2bids.")
        return

    cmd = [
        "dcm2bids", 
        "-d", os.path.join(dicom_dir, orig_id),
        "-p", anon_id,
        "-s", session,
        "-c", config,
        "-o", bids_dir,
        "--auto_extract_entities"
        
    ]

    logging.info(f"🚀 Running: {' '.join(cmd)}")

    try:
        subprocess.run(cmd, check=True)
        logging.info(f"✅ Conversion complete for {anon_id}, session {session}.")
    except subprocess.CalledProcessError as e:
        logging.error(f"❌ Conversion failed for {anon_id}, session {session}: {e}")


def run_pydeface(bids_dir, anon_id, session="1"):
    anat_dir = os.path.join(bids_dir, f"sub-{anon_id}", f"ses-{session}", "anat")
    if not os.path.exists(anat_dir):
        logging.warning(f"No anat directory for sub-{anon_id}, skipping defacing.")
        return
    for nii_file in glob.glob(os.path.join(anat_dir, "*_T1w.nii.gz")):
        cmd = ["pydeface", nii_file]
        logging.info(f"Defacing {nii_file}")
        subprocess.run(cmd, check=True)

def run_bidsphysio(orig_id, anon_id, dicom_dir, bids_dir, session):
    """
    Check for a .puls file in the participant's DICOM directory.
    Returns True if found, False if missing.
    """
    subject_dicom_path = os.path.join(dicom_dir, orig_id)

    has_puls = False
    for root, dirs, files in os.walk(subject_dicom_path):
        for f in files:
            if f.endswith(".puls"):
                has_puls = True
                logging.info(f".puls file found for subject {orig_id} at {os.path.join(root, f)}")
                break
        if has_puls:
            break

    if not has_puls:
        logging.warning(f"No .puls file found for subject {orig_id}")

    return has_puls



In [129]:
# read excel file
df = pd.read_excel(excel_mapping_file)
df = df.iloc[:, :2]  # only keep first 2 columns
df.columns = ['Alphanumerical code', 'CPP ID']

subjects = [
    name for name in os.listdir(dicom_dir)
    if os.path.isdir(os.path.join(dicom_dir, name))
]

 ## BIDS conversion

 This is the main script to do the conversion from DICOM to BIDS. Depending on the type of scans you are doing, it might take a while (approx 10-15 mins per scans)

In [130]:
not_found = 0
converted = 0
skipped = 0
missing_puls = 0

for folder_name in subjects:
    match_row = None
    folder_name_str = str(folder_name)

    for _, row in df.iterrows():
        cpp_id = str(row['CPP ID']).strip()  # Ensure string, remove spaces
        if cpp_id.startswith("CPP") and cpp_id in folder_name_str:
            match_row = row
            break

    if match_row is None:
        print(f"⚠️  No match found in Excel for {folder_name}, skipping.")
        not_found += 1
        continue

    orig_id = folder_name
    anon_id = match_row['Alphanumerical code']

    # ✅ Confirm matching before conversion
    print("\n" + "="*60)
    print(f"🧠 Found match:")
    print(f"   📁 DICOM folder : {orig_id}")
    print(f"   🧾 Assigned BIDS ID : {anon_id}")
    print(f"   🔎 Matched CPP ID : {match_row['CPP ID']}")
    print("="*60 + "\n")

    try:
        run_dcm2bids(orig_id, anon_id, dicom_dir, bids_dir, config, session)
        run_pydeface(bids_dir, anon_id, session)
        has_puls = run_bidsphysio(orig_id, anon_id, dicom_dir, bids_dir, session)

    

    except subprocess.CalledProcessError as e:
        logging.error(f"Error with subject {orig_id}: {e}")
        print(f"❌ Error with subject {orig_id}, check logs.")
        df.loc[df['CPP ID'] == match_row['CPP ID'], 'NOTES'] = f"Error: {e}"
        skipped += 1



🧠 Found match:
   📁 DICOM folder : CPP00327A
   🧾 Assigned BIDS ID : ICW26990
   🔎 Matched CPP ID : CPP00327A

INFO    | --- dcm2bids start ---
INFO    | Running the following command: /home/administrator/anaconda3/envs/dcm2bids/bin/dcm2bids -d /media/administrator/Scratch/dcm2bids-cpp/code/DICOM_ALLEN/CPP00327A -p ICW26990 -s 01 -c /media/administrator/Scratch/dcm2bids-cpp/code/dcm2bids_config.json -o /media/administrator/Scratch/dcm2bids-cpp/code/BIDS_final --auto_extract_entities
INFO    | OS version: Linux-6.8.0-85-generic-x86_64-with-glibc2.35
INFO    | Python version: 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:38:13) [GCC 12.3.0]
INFO    | dcm2bids version: 3.1.1
INFO    | dcm2niix version: v1.0.20240202
INFO    | Checking for software update
WARNING | A newer version exists for dcm2bids: 3.2.0
WARNING | You should update it -> https://github.com/UNFmontreal/Dcm2Bids.
WARNING | A newer version exists for dcm2niix: v1.0.20250506
WARNING | You should update it -> ht

# Bidsphysio conversion 


## Dry run

In [118]:

dry_run = False   # ✅ Set to True for simulation (no real conversion)


# Logging

log_file = os.path.join(bids_dir, "bidsphysio_dryrun.log")
logging.basicConfig(filename=log_file, level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
print(f"📜 Dry run logging to: {log_file}")

# Excel mapping

df = pd.read_excel(excel_mapping_file)
if not {"CPP ID", "Alphanumerical code"}.issubset(df.columns):
    raise ValueError("Excel file must contain 'CPP ID' and 'Alphanumerical code' columns")

print(f"📖 Loaded mapping for {len(df)} subjects from Excel")


#Scan and simulate conversions

to_convert = []

for root, _, files in os.walk(dicom_dir):
    for f in files:
        if f.endswith(".puls"):
            cpp_id = None
            for possible_id in df["CPP ID"]:
                if str(possible_id) in f:
                    cpp_id = str(possible_id)
                    break

            if not cpp_id:
                print(f"⚠️ No matching CPP ID found for {f}, skipping.")
                logging.warning(f"No match for {f}")
                continue

            match = df.loc[df["CPP ID"] == cpp_id, "Alphanumerical code"]
            if match.empty:
                print(f"⚠️ No anonymized ID found for {cpp_id}, skipping.")
                logging.warning(f"No anonymized ID for {cpp_id}")
                continue

            anon_id = match.values[0]
            puls_path = os.path.join(root, f)
            func_dir = os.path.join(bids_dir, f"sub-{anon_id}", f"ses-{session}", "func")

            to_convert.append({
                "CPP_ID": cpp_id,
                "Anon_ID": anon_id,
                "Puls_Path": puls_path,
                "Func_Dir": func_dir
            })

# -----------------------------
# 📊 Summary of what WOULD happen
# -----------------------------
print("\n🧪 DRY RUN SUMMARY")
print("="*60)
for i, item in enumerate(to_convert, 1):
    print(f"{i:02d}. {os.path.basename(item['Puls_Path'])}")
    print(f"   → CPP ID     : {item['CPP_ID']}")
    print(f"   → Anon ID    : {item['Anon_ID']}")
    print(f"   → Target dir : {item['Func_Dir']}")
    print("-"*60)

print(f"\n📦 Total .puls files found: {len(to_convert)}")
print(f"🧾 Log file: {log_file}")

if dry_run:
    print("\n✅ Dry run complete — no conversions executed.")
    logging.info(f"Dry run complete — {len(to_convert)} potential conversions listed.")
else:
    print("\n❌ dry_run=False — conversions will be executed (use with caution).")


📜 Dry run logging to: /media/administrator/Scratch/dcm2bids-cpp/code/BIDS_final/bidsphysio_dryrun.log
📖 Loaded mapping for 1172 subjects from Excel

🧪 DRY RUN SUMMARY
01. CPP00037B.puls
   → CPP ID     : CPP00037B
   → Anon ID    : LDG84714
   → Target dir : /media/administrator/Scratch/dcm2bids-cpp/code/BIDS_final/sub-LDG84714/ses-01/func
------------------------------------------------------------
02. CPP00038B.puls
   → CPP ID     : CPP00038B
   → Anon ID    : JYW18413
   → Target dir : /media/administrator/Scratch/dcm2bids-cpp/code/BIDS_final/sub-JYW18413/ses-01/func
------------------------------------------------------------

📦 Total .puls files found: 2
🧾 Log file: /media/administrator/Scratch/dcm2bids-cpp/code/BIDS_final/bidsphysio_dryrun.log

❌ dry_run=False — conversions will be executed (use with caution).


## Real run

In [120]:

# -----------------------------
# 🧭 Load Excel mapping
# -----------------------------
df = pd.read_excel(excel_mapping_file)
if not {"CPP ID", "Alphanumerical code"}.issubset(df.columns):
    raise ValueError("Excel file must contain 'CPP ID' and 'Alphanumerical code' columns")

print(f"📖 Loaded mapping for {len(df)} subjects from Excel")

# -----------------------------
# ⚙️ Function to run physio2bidsphysio
# -----------------------------
def run_physio2bidsphysio(puls_path, sub_id):
    func_dir = os.path.join(bids_dir, f"sub-{sub_id}", f"ses-{session}", "func")
    os.makedirs(func_dir, exist_ok=True)
    os.chdir(func_dir)

    cmd = ["physio2bidsphysio", "-i", puls_path, "-b", f"sub-{sub_id}"]
    logging.info(f"Running: {' '.join(cmd)}")
    print(f"🔄 Converting {os.path.basename(puls_path)} → sub-{sub_id}")

    try:
        subprocess.run(cmd, check=True)
        logging.info(f"✅ Converted {puls_path}")
        print(f"✅ Success for {sub_id}")
        return True
    except subprocess.CalledProcessError as e:
        logging.error(f"❌ Error converting {puls_path}: {e}")
        print(f"❌ Error for {sub_id}")
        return False

# -----------------------------
# 🚀 Main loop: Scan and convert
# -----------------------------
converted, errors, skipped = 0, 0, 0

for root, _, files in os.walk(dicom_dir):
    for f in files:
        if f.endswith(".puls"):
            cpp_id = None
            for possible_id in df["CPP ID"]:
                if str(possible_id) in f:
                    cpp_id = str(possible_id)
                    break
            if not cpp_id:
                print(f"⚠️ No matching CPP ID found in Excel for file {f}, skipping.")
                skipped += 1
                continue

            match = df.loc[df["CPP ID"] == cpp_id, "Alphanumerical code"]
            if match.empty:
                print(f"⚠️ No anonymized ID found for {cpp_id}, skipping.")
                skipped += 1
                continue

            anon_id = match.values[0]
            puls_path = os.path.join(root, f)

            success = run_physio2bidsphysio(puls_path, anon_id)
            if success:
                converted += 1
            else:
                errors += 1

# -----------------------------
# 📊 Summary
# -----------------------------
print("\n📊 Summary:")
print(f"  ✅ Converted: {converted}")
print(f"  ⚠️ Skipped (no match): {skipped}")
print(f"  ❌ Errors: {errors}")


📖 Loaded mapping for 1172 subjects from Excel
🔄 Converting CPP00037B.puls → sub-LDG84714
Unknown type of recording for /media/administrator/Scratch/dcm2bids-cpp/code/DICOM_DOUGLAS/20251024_ducsim_CPP00037B/CPP00037B.puls
We cannot save with trigger because we found no trigger.

Saving physio data

✅ Success for LDG84714
🔄 Converting CPP00038B.puls → sub-JYW18413
We cannot save with trigger because we found no trigger.

Saving physio data

✅ Success for JYW18413

📊 Summary:
  ✅ Converted: 2
  ⚠️ Skipped (no match): 0
  ❌ Errors: 0
